#Image classification using Mahotas

In [14]:
from glob import glob
import numpy as np
import mahotas as mh
import pandas as pd

In [39]:
# Load images
images = glob('data/train/train/*.png')
images = images

In [41]:
## UNCOMMENT THE BELOW LINES. BELOW LINES ARE FOR IMAGE LOADING AND FEATURE EXTRACTION
# features = []

# for im in images:
#     im = mh.imread(im)
#     im = mh.colors.rgb2gray(im, dtype=np.uint8)
#     features.append(mh.features.haralick(im).ravel())
    
# features = np.array(features)
# np.save('data/mahotas_features',features)

features = np.load('data/mahotas_features.npy') # Image data




In [37]:
"""Read labels"""
labels = pd.read_csv('data/trainLabels.csv') # Image labels
# get unique labels
unique_labels = labels.label.unique().tolist()
unique_labels = dict((val, unique_labels.index(val)) for val in unique_labels)
print unique_labels

# convert labels to int
y = [unique_labels[val] for val in labels.label]

{'horse': 5, 'automobile': 3, 'deer': 2, 'dog': 8, 'frog': 0, 'cat': 7, 'truck': 1, 'ship': 6, 'bird': 4, 'airplane': 9}


In [42]:


print '# features, labels, y', len(features), len(labels), len(y)

# features, labels, y 50000 50000 50000


In [18]:
"""Instantiate model"""
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
clf = Pipeline([('preproc', StandardScaler()),('classifier', LogisticRegression())])

In [25]:
"""Train model"""
from sklearn import cross_validation
cv = cross_validation.LeaveOneOut(len(images))
scores = cross_validation.cross_val_score(clf, features, y[:500], cv=cv)
print('Accuracy: {:.1%}'.format(scores.mean()))


Accuracy: 9.4%


In [50]:
"""Train logistic regression model"""
logr = LogisticRegression(verbose=1,solver="lbfgs",max_iter=110)
logr.fit( features[:40000], y[:40000])

[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    7.7s finished


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=110, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='lbfgs', tol=0.0001,
          verbose=1, warm_start=False)

In [51]:
results = logr.predict(features[40000:48000])

crt = 0
for i in range(len(results)):
    crt += (results[i] == y[40000+i])
    
print '# correct predictions', crt
print 'Percentage correct', (crt*100/len(results))

# correct predictions 795
Percentage correct 9


##Creating my own features